In [43]:
'''
Created on Aug. 19 2020
@author: Carl J. Raymond
'''

# Solves LOCA. Modified from GLOB

from scoring.pam250 import PAM250
from scoring.blosum62 import BLOSUM62
scorematrix = PAM250()
from fasta import read
from enum import Flag, auto

class EditOp(Flag):
    MATCH = auto()
    SUBST = auto()
    GAP_S = auto()
    GAP_T = auto()
    
    #def __repr__(self):
    #    return '<%s>' % self.name
    

def compute_cost(S, T, matrix, gap_cost):
    len_s = len(S)
    len_t = len(T)
    
    # Build the cost matrix. Each cell is a tuple: (cost, best choice EditOps)
    # with the best cost at this point, and one or more EditOps giving that cost.
    # The EditOp token MATCH means that the characters matched; SUBST
    # means they didn't match, and the choice is to substitute; GAP_S means that a
    # position in S was skipped; GAP_T means a postion in T was skipped.
    # Cost[0][j] = (j,EditOp.GAP_T) for all j and cost[i][0] = (i,EditOp.GAP_S)
    # for all i.

    # Work row-by-row, computing each position in the current row by looking
    # at the current row's previous position and the previous row.

    # Create first row of cost matrix - gap in S, consume in T.
    thisrow = [ (i*gap_cost, EditOp.MATCH if i==0 else EditOp.GAP_T) for i in range(len_s+1)]
    cost = [ thisrow ]

    # Create each new row, referring to the previous row, and append to the cost matrix
    for j in range(len_t):
        lastrow = thisrow
        thisrow = [ ((j+1)*gap_cost, EditOp.GAP_S) ]
        for i in range(len_s):

            prevscore = lastrow[i][0]
            bestop = (prevscore + matrix.score(S[i], T[j]), EditOp.MATCH)

            # Compare other operations. Accumulate
            # all possibilities that have the same minimum cost.

            # Gap in S, consume in T
            gap_s = lastrow[i+1][0] + gap_cost
            if gap_s > bestop[0]:
                # Best so far
                bestop = (gap_s, EditOp.GAP_S)
            elif gap_s == bestop[0]:
                # Same score as current best. Accumulate EditOp.
                bestop = (bestop[0], bestop[1] | EditOp.GAP_S)

            # Gap in T, consume in S
            gap_t = thisrow[i][0] + gap_cost
            if gap_t > bestop[0]:
                # Best so far
                bestop = (gap_t, EditOp.GAP_T)
            elif gap_t == bestop[0]:
                # Same score as current best. Accumulate EditOp.
                bestop = (bestop[0], bestop[1] | EditOp.GAP_T)

            thisrow.append(bestop)

        cost.append(thisrow)

    return cost

def best_position(matrix):
    # Find the position with the highest score
    bestindex = (0,0)
    bestscore = 0
    for j,row in enumerate(matrix):
        for i,elem in enumerate(row):
            if elem[0] > bestscore:
                bestscore = elem[0]
                bestindex = (i,j)
    return bestindex, bestscore
    
with open("data/rosalind_loca.txt") as spec:
    data_s, data_t = read(spec)

_, S = data_s
#S = "PLEASANTLY"
#S = "MEANLY"
#S = "MEANLYPRTEINSTRING"

_, T = data_t    
#T = "MEANLY"
#T = "PLEASANTLY"
#T = "PLEASANTLYEINSTEIN"

print(f"S (length {len(S)}): {S}")
print(f"T (length {len(T)}): {T}")

cost = compute_cost(S, T, PAM250(), -5)

#for row in cost:
#    for k in row:
#        print(f"{k[0]:>4} ", end='')
#    print()

# The final edit cost is down in the corner
dist = cost[-1][-1][0]
print(f"Edit distance: {dist}")

# Find the position with the highest score
bestpos, bestscore = best_position(cost)
print(f"Best index: {bestpos}={bestscore}")

# Truncate and reverse strings at bestindex
print("Matching truncated reversed strings:")
trunc_S = S[bestpos[0]-1::-1]
print(trunc_S)
trunc_T = T[bestpos[1]-1::-1]
print(trunc_T)

cost2 = compute_cost(trunc_S, trunc_T, PAM250(), -5)

#for row in cost2:
#    for k in row:
#        print(f"{k[0]: >4} ", end='')
#    print()

# Find the position with the highest score
bestpos, bestscore = best_position(cost2)
print(f"Best index: {bestpos}={bestscore}")

substring_S = trunc_S[bestpos[0]-1::-1]
substring_T = trunc_T[bestpos[1]-1::-1]

print("Optimal substrings:")
print(substring_S)
print(substring_T)

# Walk the cost matrix from the far corner back to the origin
# recording a lowest cost alignment sequences in reverse. This
# is one optimal alignment out of many.
revalign_S = []
revalign_T = []
revtrace = []
i, j = len(substring_S), len(substring_T)
while i>0 or j>0:

    step = cost2[j][i]
    action = step[1]
    if action & (EditOp.MATCH | EditOp.SUBST):
        # Consume in S and T.
        i -= 1
        revalign_S.append(substring_S[i])
        j -= 1
        revalign_T.append(substring_T[j])
            
    elif action & EditOp.GAP_S:
        # Gap in S, consume in T
        revalign_S.append('-')
        j -= 1
        revalign_T.append(substring_T[j])
    elif action & EditOp.GAP_T:
        # Gap in T, consume in S
        i -= 1
        revalign_S.append(substring_S[i])
        revalign_T.append('-')

    tracechar = '*' if substring_S[i] == substring_T[j] else ' '
    revtrace.append(tracechar)
    
align_S = "".join(revalign_S[::-1])
align_T = "".join(revalign_T[::-1])
trace = "".join(revtrace[::-1])

#print(align_s)
#print(align_t)
#print(trace)
with open("data/rosalind_loca.out", "w+") as output:
    output.write(f"Optimal distance: {bestscore}\n")
    output.write("{0}\n".format(substring_S))
    output.write("{0}\n".format(substring_T))
    output.write("\n")
    output.write("{0}\n".format(align_S))
    output.write("{0}\n".format(align_T))
    output.write("{0}\n".format(trace))


S (length 900): FPELCCWSWVHLAEMDRMCAMFVGFTFPMAQCRAPNITKICFLCLLWMDSVFVLPAHKHTRKLWSLTMGYCCCEHTGYYQMMDCREPQYLVSCNIMAWHDDSPGFTFYTNRKEMHHEPKPWVQFIFNTSGPQNPGHAFLCAKAIHWQDWRLGWNGYVMEGQSRYYEPMADPKFGQGCKPTKGQTIRRLPNDSGEWRGGKFHMCGTIEWRTPQMAQGSVAKETMVTKGAKTTQWMRGEHVIRSTCSNYKWEPMHCSVCTCCFHWFMDFCLNRMNNCFWPIGDEFDRDQAVPMWAKWWFDLQIYPHHIQCWPRSSQPAENFMYIGPLVTWDLTWHADVCGNCECMSHLQISNAYSHSASEVSQAYTPFVIHHMNWRKVRGLNEGIPKCWGVCTDEDSPFLIGALCSICCMQVACVPIRIPYHPKHIPLSVVYRKTHWLYNNKHPMPCLTDFQWNMWAYSQIVHSQNDIDCVSHPWNWVTHDHTYQWAFDDHNSWLQVKFYADTLDTAAVRHGGGILVMAELVRSKGDVWMTNECLKVRYAMSQNGTNGTGAMYCCRWDMDNDTHEMKFTPASDKKRFETKCLWADLLTWAPKDADIKCKWVDQMFRGDPKMRNHNSTHGCKDKEDAYLYSPQTRDIRFSGSVQTEVPAPWCQWMHPYNDIQSRLWQWNDLPYENGTMILPTQKLVQDVNQAYIKKFFELPFIARWLIGHPVLKRKKMTVWQLWFDKHWRLACKNWGFSLRKFVFMVGNTWAGVQPKHDWCNGAHKDLPLTFWVPGIHHLFCWSSLCKHFIMNPVQQCVCCITVSITLLFVPRYKVHCCAFLEQCMGSLKFQCVVESLGTPNCIVPDLNRAKSHYTALYKQDDHLISKINGEMFENWRCCTGCPNAGFWHTCNTRMVTRSWRR
T (length 937): HDASGRKPYDLFPKASTSGGLRMAGNQKKSRRGMFHVIVPIQWRTDGEDIPSVDHLGGSATKPMFST